In [ ]:
# notebooks/3d_extract_pdf_pairs_hybrid.ipynb (새 노트북 예시)

import os
import sys
import pandas as pd
# src 폴더를 경로에 추가
sys.path.append(os.path.abspath('../src'))


In [ ]:

# ----------------------------------------------------------------
# 1. 설정
# ----------------------------------------------------------------
# config.py에서 필요한 경로들을 불러옵니다.
from config import PDF_DOWNLOAD_DIR, PROCESSED_DIR
# 새로 만든 모듈을 불러옵니다.
from processors.hybrid_pdf_pair_extractor import HybridPDFPairExtractor

# 처리할 PDF 파일명을 지정합니다.
SOURCE_PDF_FILENAME = '10x-gene-expression-tech-note-m-gl-00777.pdf'
TARGET_PDF_FILENAME = '10x-gene-expression-tech-note-m-gl-00777-jpn.pdf'

# config의 경로와 파일명을 조합하여 전체 경로를 만듭니다.
source_pdf_path = os.path.join(PDF_DOWNLOAD_DIR, 'en', SOURCE_PDF_FILENAME)
target_pdf_path = os.path.join(PDF_DOWNLOAD_DIR, 'ja', TARGET_PDF_FILENAME)

# 결과 파일 경로
output_csv_path = os.path.join(PROCESSED_DIR, 'pdf_tm_en_ja_hybrid_output.csv')


In [ ]:

# ----------------------------------------------------------------
# 2. 실행
# ----------------------------------------------------------------
if not os.path.exists(source_pdf_path) or not os.path.exists(target_pdf_path):
    print(f"❌ 오류: PDF 파일을 찾을 수 없습니다. 경로를 확인하세요.")
    print(f"   - 원본(EN) 경로: {source_pdf_path}")
    print(f"   - 대상(JA) 경로: {target_pdf_path}")
else:
    # 클래스 인스턴스 생성
    extractor = HybridPDFPairExtractor()
    
    # 번역쌍 생성 실행 (x_tolerance 추가)
    df_pairs = extractor.generate_pairs(
        source_pdf_path=source_pdf_path,
        target_pdf_path=target_pdf_path,
        y_tolerance=10,         # 수직 허용 오차 (10픽셀)
        x_tolerance=50,         # ◀◀◀ 수평 허용 오차 (50픽셀) 추가
        similarity_threshold=0.75 # ◀◀◀ 정확도를 위해 임계값 상향 조정 추천
    )

    # ----------------------------------------------------------------
    # 3. 결과 저장 및 요약
    # ----------------------------------------------------------------
    if not df_pairs.empty:
        print("\n--- 4. 결과 저장 ---")
        df_pairs.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        
        print(f"   - ✅ 총 {len(df_pairs)}개의 번역 쌍을 '{os.path.basename(output_csv_path)}' 파일로 저장했습니다.")
        print(f"   - 📊 평균 유사도: {df_pairs['similarity_score'].mean():.4f}")
        print(f"   - 📋 타입별 개수:\n{df_pairs['type'].value_counts().to_string()}")
        
        # 결과 미리보기
        print("\n--- 5. 상위 5개 결과 미리보기 (유사도 순) ---")
        print(df_pairs.nlargest(5, 'similarity_score').to_string())
    else:
        print("\n⚠️ 생성된 번역 쌍이 없습니다.")